# Model Prediction
This notebook implements prediction using sliding window approach.

## Imports and Setup

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
data_dir = "../data/"
output_dir = "../outputs/"
input_file = data_dir + '20200326_dataset.csv'

pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'
indices_dir = data_dir + 'indices/'
tmp_dir = data_dir + 'tmp/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

areas = ['maicao', 'riohacha', 'uribia']

## Load Dataset

In [3]:
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (334524, 112)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.15970,0.13735,0.1531,0.18700,0.2090,0.26320,0.30515,0.26965,0.3327,0.0411,...,0.187614,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,3,0
1,0.15970,0.13905,0.1454,0.17845,0.2090,0.26320,0.30515,0.26395,0.3327,0.0411,...,0.177058,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,3,0
2,0.16675,0.14875,0.1589,0.18605,0.2258,0.27945,0.32070,0.28085,0.3452,0.0416,...,0.179191,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,3,0


## Resample Dataset
Resamples 30,000 negative examples per area.

In [4]:
data = model_utils.resample(data, num_neg_samples=30000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (104556, 112)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.22620,0.18300,0.1790,0.1831,0.18860,0.2509,0.2981,0.2653,0.3291,0.0431,...,0.231549,-0.594230,0.211756,0.102407,0.321400,-0.798628,-0.067515,1.196927,3,0
1,0.14955,0.12935,0.1270,0.1223,0.13935,0.2424,0.2846,0.2662,0.3236,0.0388,...,0.219911,-0.529955,0.153344,0.208832,0.384915,-0.705785,-0.057656,0.871430,3,0
2,0.16860,0.15970,0.1612,0.1616,0.18550,0.3246,0.3985,0.3127,0.4143,0.0454,...,0.205571,-0.536190,0.136678,0.237628,0.413957,-0.706110,-0.052031,0.864904,3,0


## Machine Learning Pipeline

In [5]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
SEED = 42

## Define Features and Target Label

In [6]:
# Get target and feature columns
label = 'target'
features = [column for column in data.columns[:-2] if 'mndwi' not in column]

# Convert target labels to binary
data[label] = data[label].replace({2:0, 3:0})

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (104556, 112)
Class distribution:
0    90000
1    14556
Name: target, dtype: int64


## Define Best Feature Set

In [7]:
lsvc_best_features = [
    'B2_2016', 'B11_2016', 'B12_2016', 'nbi_2016', 'mbi_2016', 'B1_2017', 'B5_2017', 
    'B11_2017', 'ui_2017', 'B10_2018', 'ui_2018', 'nbai_2018', 'mbi_2018', 'B2_2019', 
    'B6_2019', 'B7_2019', 'B9_2019', 'B10_2019', 'B12_2019', 'ndbi_2019', 'savi_2019', 
    'nbi_2019', 'brba_2019', 'mbi_2019', 'B1_2020', 'B4_2020', 'B6_2020', 'B9_2020', 
    'B12_2020', 'savi_2020'
]

## Train Model

In [8]:
lsvc = LinearSVC(C=100, random_state=SEED)

X = data[lsvc_best_features].fillna(0)
y = data[label]

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

lsvc.fit(X, y);

## Prediction: Memory-intensive Initial Approach

In [9]:
#pd.set_option('use_inf_as_na', True)
#area_dict = geoutils.get_filepaths(areas, sentinel_dir, indices_dir, pos_mask_dir, neg_mask_dir)
#data = geoutils.read_bands(area_dict, 'uribia')
#print('Data dimensions: {}'.format(data.shape))
#data.head(3)

In [10]:
#preds = lsvc._predict_proba_lr(data[lsvc_best_features].fillna(0))[:, 1]
#preds[(data.sum(axis=1) == 0)] = -1

#geoutils.save_predictions(
#    preds, 
#    image_src=area_dict['uribia']['images_cropped'][0], 
#    output_file=data_dir+'uribia_initial.tiff'
#)

## Prediction: Sliding Window Approach

In [14]:
area_dict = geoutils.get_filepaths(areas, sentinel_dir, pos_mask_dir, neg_mask_dir)
grid_blocks = 9

for area in areas:
    output = output_dir + '20200331_{}_thresh60.tif'.format(area)
    geoutils.get_preds_windowing(
        area=area, 
        area_dict=area_dict,
        model=lsvc, 
        tmp_dir=tmp_dir,
        best_features=lsvc_best_features, 
        scaler=scaler, 
        output=output, 
        grid_blocks=grid_blocks,
        threshold=0.60
    )

  0%|          | 0/81 [00:00<?, ?it/s]

Reading maicao...


100%|██████████| 81/81 [09:04<00:00,  6.72s/it]


Saving to ../outputs/20200331_maicao_thresh60.tif...
Stitching all rasters into one


  0%|          | 0/81 [00:00<?, ?it/s]

...

Saving to ../outputs/20200331_riohacha_thresh60.tif...
Stitching all rasters into one


  0%|          | 0/81 [00:00<?, ?it/s]

Reading uribia...


100%|██████████| 81/81 [03:34<00:00,  2.65s/it]


Saving to ../outputs/20200331_uribia_thresh60.tif...
Stitching all rasters into one


In [15]:
for area in areas:
    filename = output_dir + '20200331_{}_thresh60.tif'.format(area)
    !gsutil -q cp {filename} gs://immap-output/